In [12]:
pip install numpy

     --------------------------------------- 15.9/15.9 MB 50.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\ctama\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


In [15]:
pip install hrv-analysis

     ---------------------------------------- 40.6/40.6 KB 2.0 MB/s eta 0:00:00
     ---------------------------------------- 6.4/6.4 MB 33.8 MB/s eta 0:00:00
     ------------------------------------- 225.1/225.1 KB 13.4 MB/s eta 0:00:00
     --------------------------------------- 46.2/46.2 MB 40.9 MB/s eta 0:00:00
     ---------------------------------------- 7.8/7.8 MB 62.5 MB/s eta 0:00:00
     --------------------------------------- 11.6/11.6 MB 59.9 MB/s eta 0:00:00
     --------------------------------------- 15.8/15.8 MB 50.4 MB/s eta 0:00:00
     -------------------------------------- 162.3/162.3 KB 9.5 MB/s eta 0:00:00
     ---------------------------------------- 2.0/2.0 MB 62.7 MB/s eta 0:00:00
     ------------------------------------- 349.4/349.4 KB 21.2 MB/s eta 0:00:00
     ---------------------------------------- 111.1/111.1 KB ? eta 0:00:00
     ---------------------------------------- 2.7/2.7 MB 56.7 MB/s eta 0:00:00
     -------------------------------------- 211.8

You should consider upgrading via the 'C:\Users\ctama\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


In [1]:
pip install neurokit2

     -------------------------------------- 693.1/693.1 KB 5.5 MB/s eta 0:00:00
     ---------------------------------------- 64.9/64.9 KB ? eta 0:00:00
     --------------------------------------- 11.2/11.2 MB 54.7 MB/s eta 0:00:00
     ------------------------------------- 301.8/301.8 KB 19.4 MB/s eta 0:00:00
     ---------------------------------------- 128.7/128.7 KB ? eta 0:00:00
     ---------------------------------------- 70.4/70.4 KB ? eta 0:00:00
     ---------------------------------------- 166.4/166.4 KB ? eta 0:00:00
     -------------------------------------- 102.8/102.8 KB 5.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\ctama\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


In [6]:
pip install openpyxl

     -------------------------------------- 250.9/250.9 KB 3.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\ctama\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


Subimos la base de datos que ya tenemos procesada

In [9]:
import pandas as pd
# Ruta del archivo Excel
ruta_archivo = r"C:\Users\ctama\OneDrive\Desktop\Bioseñales\Proyecto\Analisis 2\DatosfiltradosconpicosR.xlsx"  # También puede ser .xls

# Leer el archivo
df_balanceado = pd.read_excel(ruta_archivo)

# Mostrar las primeras filas
print(df_balanceado.head())

                     FileName Rhythm      Beat  PatientAge Gender  \
0  MUSE_20180712_160452_75000   AFIB       LVH          74   MALE   
1  MUSE_20180118_131905_87000   AFIB       TWC          72   MALE   
2  MUSE_20180114_071444_05000   AFIB       TWC          74   MALE   
3  MUSE_20180114_132956_24000   AFIB  LVHV TWC          68   MALE   
4  MUSE_20180118_124852_18000   AFIB      STTU          78   MALE   

   VentricularRate  AtrialRate  QRSDuration  QTInterval  QTCorrected  ...  \
0               49          57           92         488          440  ...   
1              151         170           84         298          472  ...   
2              107         108           82         342          456  ...   
3               95          87           84         334          419  ...   
4               97         153          104         344          436  ...   

   QRSCount  QOnset  QOffset  TOffset  AgeGroup        SDNN       RMSSD  \
0         7     221      267      465     61-80

Verificamos el codigo para una sola señal

In [ ]:
from hrvanalysis import get_time_domain_features
import numpy as np
import pandas as pd
import neurokit2 as nk
from hrvanalysis import remove_outliers, interpolate_nan_values, get_time_domain_features

# === Configuración ===
indice = 0  # Cambia este índice al que desees
nombre_archivo = df_balanceado['FileName'][indice]
frecuencia_muestreo = 500  # Hz

# === Cargar archivo ===
ruta = rf'C:\Users\ctama\OneDrive\Desktop\Bioseñales\Proyecto\Datos\ECGDataDenoised\ECGDataDenoised\{nombre_archivo}.csv'
senal = pd.read_csv(ruta, delimiter=',', names=['I', 'II', 'III', 'aVR', 'aVL', 'aVF', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6'])

# === Verificar y extraer derivación II ===
if "II" not in senal.columns:
    raise ValueError("La columna 'II' no existe en el archivo.")
ecg_signal = senal['II'].values

# === Detectar picos R con NeuroKit2 ===
_, info = nk.ecg_process(ecg_signal, sampling_rate=frecuencia_muestreo)
picos_r = info["ECG_R_Peaks"]

# === Calcular RR intervals (en ms para hrvanalysis) ===
rr_intervals = np.diff(picos_r) / frecuencia_muestreo * 1000  # Convertir a milisegundos

# === Preprocesamiento con hrvanalysis ===
rr_clean = remove_outliers(rr_intervals, low_rri=300, high_rri=2000)
rr_clean = interpolate_nan_values(rr_clean)

# === Calcular varianza con hrvanalysis ===
time_features = get_time_domain_features(rr_clean)


0 outlier(s) have been deleted.


In [14]:
time_features

{'mean_nni': 1062.5,
 'sdnn': 361.640310648183,
 'sdsd': 390.3651874742889,
 'nni_50': 6,
 'pnni_50': 85.71428571428571,
 'nni_20': 6,
 'pnni_20': 85.71428571428571,
 'rmssd': 409.32767174617317,
 'median_nni': 1171.0,
 'range_nni': 1214.0,
 'cvsd': 0.38524957340816296,
 'cvnni': 0.34036735119828987,
 'mean_hr': 67.9829595507361,
 'max_hr': 169.4915254237288,
 'min_hr': 38.265306122448976,
 'std_hr': 39.56801701612557}

Realizamos el procesamiento para toda la base de datos (7480 datos), ademas generamos 2 database, una con solo los resultados del hvanalysis, y la otra con la base de datos ya incorporada

In [17]:
import pandas as pd
import numpy as np
import neurokit2 as nk
from hrvanalysis import remove_outliers, interpolate_nan_values, get_time_domain_features


# Simulador de función para cargar archivos (en tu código real, esta parte ya debe estar bien configurada)
def cargar_senal(nombre_archivo):
    # Aquí simularíamos una señal sintética, pero en tu código leerías el CSV real
    return nk.ecg_simulate(duration=10, sampling_rate=500)  # Simulación de 10 segundos

# Configuración
frecuencia_muestreo = 500  # Hz

# Lista para almacenar los resultados
resultados = []

for nombre_archivo in df_balanceado['FileName']:
    # Cargar señal (en tu código real, sería pd.read_csv con la ruta)
    ecg_signal = cargar_senal(nombre_archivo)
    
    # Detectar picos R con NeuroKit2
    _, info = nk.ecg_process(ecg_signal, sampling_rate=frecuencia_muestreo)
    picos_r = info["ECG_R_Peaks"]
    
    # Calcular intervalos RR (en ms para hrvanalysis)
    rr_intervals = np.diff(picos_r) / frecuencia_muestreo * 1000  # ms
    
    # Preprocesar RR para hrvanalysis
    rr_clean = remove_outliers(rr_intervals)
    rr_clean = interpolate_nan_values(rr_clean)
    
    # Extraer todas las métricas de dominio del tiempo
    time_features = get_time_domain_features(rr_clean)
    time_features["FileName"] = nombre_archivo  # Agregamos el nombre del archivo
    
    resultados.append(time_features)

# Convertir lista de resultados a DataFrame
df_resultados = pd.DataFrame(resultados)

# Unir los resultados a df_balanceado por FileName
df_balanceado = df_balanceado.merge(df_resultados, on='FileName', how='left')

#df_balanceado.head()

0 outlier(s) have been deleted.
0 outlier(s) have been deleted.
0 outlier(s) have been deleted.
0 outlier(s) have been deleted.
0 outlier(s) have been deleted.
0 outlier(s) have been deleted.
0 outlier(s) have been deleted.
0 outlier(s) have been deleted.
0 outlier(s) have been deleted.
0 outlier(s) have been deleted.
0 outlier(s) have been deleted.
0 outlier(s) have been deleted.
0 outlier(s) have been deleted.
0 outlier(s) have been deleted.
0 outlier(s) have been deleted.
0 outlier(s) have been deleted.
0 outlier(s) have been deleted.
0 outlier(s) have been deleted.
0 outlier(s) have been deleted.
0 outlier(s) have been deleted.
0 outlier(s) have been deleted.
0 outlier(s) have been deleted.
0 outlier(s) have been deleted.
0 outlier(s) have been deleted.
0 outlier(s) have been deleted.
0 outlier(s) have been deleted.
0 outlier(s) have been deleted.
0 outlier(s) have been deleted.
0 outlier(s) have been deleted.
0 outlier(s) have been deleted.
0 outlier(s) have been deleted.
0 outlie

In [19]:
df_resultados

,mean_nni,sdnn,sdsd,nni_50,pnni_50,nni_20,pnni_20,rmssd,median_nni,range_nni,cvsd,cvnni,mean_hr,max_hr,min_hr,std_hr,FileName
0,856.0,12.110601,15.663120,0,0.0,2,22.222222,15.677301,856.0,38.0,0.018315,0.014148,70.106099,71.770335,68.649886,0.941932,MUSE_20180712_160452_75000
1,855.8,9.542886,12.511723,0,0.0,2,22.222222,12.543258,854.0,32.0,0.014657,0.011151,70.117706,71.599045,68.965517,0.743718,MUSE_20180118_131905_87000
2,856.8,12.407883,15.463218,0,0.0,1,11.111111,15.691470,857.0,40.0,0.018314,0.014482,70.041350,72.115385,68.807339,0.971038,MUSE_20180114_071444_05000
3,857.2,11.477514,14.110674,0,0.0,1,11.111111,14.360439,856.0,40.0,0.016753,0.013390,70.006535,71.428571,68.181818,0.881921,MUSE_20180114_132956_24000
4,857.4,15.550634,20.066556,0,0.0,4,44.444444,20.066556,854.0,42.0,0.023404,0.018137,69.999671,71.599045,68.181818,1.201117,MUSE_20180118_124852_18000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7475,855.8,13.934769,17.048949,0,0.0,2,22.222222,17.101007,852.0,40.0,0.019982,0.016283,70.126508,71.599045,68.337130,1.079221,MUSE_20180114_124959_72000
7476,857.8,15.273797,19.888579,0,0.0,4,44.444444,19.899749,859.0,42.0,0.023199,0.017806,69.966390,71.770335,68.337130,1.185039,MUSE_20180113_134050_13000
7477,856.8,8.121303,8.697949,0,0.0,0,0.000000,9.165151,857.0,28.0,0.010697,0.009479,70.033670,71.090047,68.807339,0.629361,MUSE_20180114_071212_35000
7478,856.2,10.643725,10.877545,0,0.0,0,0.000000,11.313708,854.0,38.0,0.013214,0.012431,70.086846,71.770335,68.649886,0.827815,MUSE_20180115_123937_31000


In [20]:
df_resultados.to_excel('DatosFinalHRV_Analysis.xlsx', index=False)

In [21]:
df_balanceado2 = pd.read_excel(ruta_archivo)

In [26]:
df_balanceado

,FileName,Rhythm,Beat,PatientAge,Gender,VentricularRate,AtrialRate,QRSDuration,QTInterval,QTCorrected,...,pnni_20_y,rmssd_y,median_nni_y,range_nni_y,cvsd_y,cvnni_y,mean_hr_y,max_hr_y,min_hr_y,std_hr_y
0,MUSE_20180712_160452_75000,AFIB,LVH,74,MALE,49,57,92,488,440,...,22.222222,15.677301,856.0,38.0,0.018315,0.014148,70.106099,71.770335,68.649886,0.941932
1,MUSE_20180118_131905_87000,AFIB,TWC,72,MALE,151,170,84,298,472,...,22.222222,12.543258,854.0,32.0,0.014657,0.011151,70.117706,71.599045,68.965517,0.743718
2,MUSE_20180114_071444_05000,AFIB,TWC,74,MALE,107,108,82,342,456,...,11.111111,15.691470,857.0,40.0,0.018314,0.014482,70.041350,72.115385,68.807339,0.971038
3,MUSE_20180114_132956_24000,AFIB,LVHV TWC,68,MALE,95,87,84,334,419,...,11.111111,14.360439,856.0,40.0,0.016753,0.013390,70.006535,71.428571,68.181818,0.881921
4,MUSE_20180118_124852_18000,AFIB,STTU,78,MALE,97,153,104,344,436,...,44.444444,20.066556,854.0,42.0,0.023404,0.018137,69.999671,71.599045,68.181818,1.201117
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7475,MUSE_20180114_124959_72000,AT,LVHV STDD,77,FEMALE,110,110,106,310,419,...,22.222222,17.101007,852.0,40.0,0.019982,0.016283,70.126508,71.599045,68.337130,1.079221
7476,MUSE_20180113_134050_13000,AT,NONE,79,FEMALE,109,109,72,350,471,...,44.444444,19.899749,859.0,42.0,0.023199,0.017806,69.966390,71.770335,68.337130,1.185039
7477,MUSE_20180114_071212_35000,AT,2AVB STTC,75,FEMALE,143,156,74,328,506,...,0.000000,9.165151,857.0,28.0,0.010697,0.009479,70.033670,71.090047,68.807339,0.629361
7478,MUSE_20180115_123937_31000,AT,NONE,63,FEMALE,99,99,76,348,446,...,0.000000,11.313708,854.0,38.0,0.013214,0.012431,70.086846,71.770335,68.649886,0.827815


In [27]:
df_resultados

,mean_nni,sdnn,sdsd,nni_50,pnni_50,nni_20,pnni_20,rmssd,median_nni,range_nni,cvsd,cvnni,mean_hr,max_hr,min_hr,std_hr,FileName
0,856.0,12.110601,15.663120,0,0.0,2,22.222222,15.677301,856.0,38.0,0.018315,0.014148,70.106099,71.770335,68.649886,0.941932,MUSE_20180712_160452_75000
1,855.8,9.542886,12.511723,0,0.0,2,22.222222,12.543258,854.0,32.0,0.014657,0.011151,70.117706,71.599045,68.965517,0.743718,MUSE_20180118_131905_87000
2,856.8,12.407883,15.463218,0,0.0,1,11.111111,15.691470,857.0,40.0,0.018314,0.014482,70.041350,72.115385,68.807339,0.971038,MUSE_20180114_071444_05000
3,857.2,11.477514,14.110674,0,0.0,1,11.111111,14.360439,856.0,40.0,0.016753,0.013390,70.006535,71.428571,68.181818,0.881921,MUSE_20180114_132956_24000
4,857.4,15.550634,20.066556,0,0.0,4,44.444444,20.066556,854.0,42.0,0.023404,0.018137,69.999671,71.599045,68.181818,1.201117,MUSE_20180118_124852_18000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7475,855.8,13.934769,17.048949,0,0.0,2,22.222222,17.101007,852.0,40.0,0.019982,0.016283,70.126508,71.599045,68.337130,1.079221,MUSE_20180114_124959_72000
7476,857.8,15.273797,19.888579,0,0.0,4,44.444444,19.899749,859.0,42.0,0.023199,0.017806,69.966390,71.770335,68.337130,1.185039,MUSE_20180113_134050_13000
7477,856.8,8.121303,8.697949,0,0.0,0,0.000000,9.165151,857.0,28.0,0.010697,0.009479,70.033670,71.090047,68.807339,0.629361,MUSE_20180114_071212_35000
7478,856.2,10.643725,10.877545,0,0.0,0,0.000000,11.313708,854.0,38.0,0.013214,0.012431,70.086846,71.770335,68.649886,0.827815,MUSE_20180115_123937_31000


In [28]:
dfmerge = df_balanceado.merge(df_resultados, on='FileName', how='left')

In [30]:
dfmerge.to_excel('DatosFinal_HRV_Analysis.xlsx', index=False)